In [1]:
import sys
sys.path.insert(0, '../training')
from loader import EOS_TOKEN, PAD_TOKEN
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sentencepiece as spm
from tqdm import tqdm
import json
from scipy.stats import pearsonr
import copy
import random

protein_mapping = {
    'D': [0, 0, 0, 1, 1, 1, 1, 0, 0],
    'E': [0, 0, 1, 0, 1, 1, 1, 0, 0],
    'K': [0, 0, 1, 1, 0, 1, 1, 0, 0],
    'R': [0, 0, 1, 1, 1, 0, 1, 0, 0],
    'H': [0, 1, 0, 0, 1, 1, 1, 0, 0],
    'S': [0, 1, 0, 1, 0, 1, 0, 1, 0],
    'T': [0, 1, 0, 1, 1, 0, 0, 1, 0],
    'N': [0, 1, 1, 0, 0, 1, 0, 1, 0],
    'Q': [0, 1, 1, 0, 1, 0, 0, 1, 0],
    'Y': [0, 1, 1, 1, 0, 0, 0, 1, 0],
    'G': [1, 0, 0, 0, 1, 1, 0, 0, 1],
    'A': [1, 0, 0, 1, 0, 1, 0, 0, 1],
    'V': [1, 0, 0, 1, 1, 0, 0, 0, 1],
    'L': [1, 0, 1, 0, 0, 1, 0, 0, 1],
    'I': [1, 0, 1, 0, 1, 0, 0, 0, 1],
    'M': [1, 0, 1, 1, 0, 0, 0, 0, 1],
    'F': [1, 1, 0, 0, 0, 1, 0, 0, 1],
    'W': [1, 1, 0, 0, 1, 0, 0, 0, 1],
    'P': [1, 1, 0, 1, 0, 0, 0, 0, 1],
    'C': [1, 1, 1, 0, 0, 0, 0, 0, 1]
}

nucleotide_mapping = {
    'A': [1, 0, 0, 0, 0],
    'C': [0, 1, 0, 0, 0],
    'G': [0, 0, 1, 0, 0],
    'T': [0, 0, 0, 1, 0],
    'U': [0, 0, 0, 0, 1]
}

device = "cuda"
dtype = torch.float32

In [2]:
class DeePNAP(nn.Module):
    def __init__(self):
        super(DeePNAP, self).__init__()
        self.protein_conv1 = nn.Conv2d(1, 48, (6, 9), stride=(6, 1))
        self.protein_conv2 = nn.Conv2d(1, 48, (6, 9), stride=(6, 1))

        self.dna_conv1 = nn.Conv2d(1, 32, (2, 5), stride=(2, 1))
        self.dna_conv2 = nn.Conv2d(1, 32, (2, 5), stride=(2, 1))

        self.interaction_layer1_p1_n1 = nn.Linear(168 + 39, 96)
        self.interaction_layer2_p1_n1 = nn.Linear(96, 32)

        self.interaction_layer1_p1_n2 = nn.Linear(168 + 39, 96)
        self.interaction_layer2_p1_n2 = nn.Linear(96, 32)

        self.interaction_layer1_p2_n1 = nn.Linear(168 + 39, 96)
        self.interaction_layer2_p2_n1 = nn.Linear(96, 32)

        self.interaction_layer1_p2_n2 = nn.Linear(168 + 39, 96)
        self.interaction_layer2_p2_n2 = nn.Linear(96, 32)

        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)

        self.dropout = nn.Dropout(0.5)

    def forward(self, protein, nucleotide):
        p1 = F.relu(self.protein_conv1(F.pad(protein, (0, 0, 0, 8))))
        p1 = p1.transpose(3, 1)
        p1 = F.max_pool2d(p1, (1, 48))
        p1 = p1.flatten(start_dim=1)

        # pad protein with 
        p2 = F.relu(self.protein_conv2(F.pad(protein, (0, 0, 8, 0))))
        p2 = p2.transpose(3, 1)
        p2 = F.max_pool2d(p2, (1, 48))
        p2 = p2.flatten(start_dim=1)

        n1 = F.relu(self.dna_conv1(F.pad(nucleotide, (0, 0, 0, 3))))
        n1 = n1.transpose(3, 1)
        n1 = F.max_pool2d(n1, (1, 32))
        n1 = n1.flatten(start_dim=1)

        n2 = F.relu(self.dna_conv2(F.pad(nucleotide, (0, 0, 3, 0))))
        n2 = n2.transpose(3, 1)
        n2 = F.max_pool2d(n2, (1, 32))
        n2 = n2.flatten(start_dim=1)

        p1_n1 = torch.cat((p1, n1), 1)
        p1_n2 = torch.cat((p1, n2), 1)
        p2_n1 = torch.cat((p2, n1), 1)
        p2_n2 = torch.cat((p2, n2), 1)

        p1_n1 = F.leaky_relu(self.interaction_layer1_p1_n1(p1_n1))
        p1_n1 = F.leaky_relu(self.interaction_layer2_p1_n1(p1_n1))

        p1_n2 = F.leaky_relu(self.interaction_layer1_p1_n2(p1_n2))
        p1_n2 = F.leaky_relu(self.interaction_layer2_p1_n2(p1_n2))

        p2_n1 = F.leaky_relu(self.interaction_layer1_p2_n1(p2_n1))
        p2_n1 = F.leaky_relu(self.interaction_layer2_p2_n1(p2_n1))

        p2_n2 = F.leaky_relu(self.interaction_layer1_p2_n2(p2_n2))
        p2_n2 = F.leaky_relu(self.interaction_layer2_p2_n2(p2_n2))

        x = torch.cat((p1_n1, p1_n2, p2_n1, p2_n2), 1)

        res = F.leaky_relu(self.fc1(x))
        res = self.dropout(res)
        res = F.leaky_relu(self.fc2(res))
        res = self.dropout(res)
        x = x + res
        x = F.leaky_relu(self.fc3(x))
        x = self.fc4(x)

        return x

In [3]:
def process_dataset(dataset, noise_floor=1e-15):
    nucleotide_sequences = []
    peptides = []

    Kd = []
    G0 = []

    for key in dataset:
        for item in dataset[key]["binding data"]:
            if item[2] == 0 or item[3] == 0:
                continue
            nucleotide_sequence = item[0]
            if item[1] == "RNA":
                nucleotide_sequence = "<RNA>" + nucleotide_sequence + "<EOS>"
            else:
                nucleotide_sequence = "<DNA>" + nucleotide_sequence + "<EOS>"
            
            nucleotide_sequences.append(nucleotide_sequence)
            peptides.append(dataset[key]['Sequence']) # we exclude the tags here because they are added in prepare_sample after mutation

            Kd.append(np.log10(item[2] + noise_floor*np.random.uniform() + noise_floor))
            G0.append(item[3])

    return nucleotide_sequences, peptides, Kd, G0

def encode_peptide(peptide):
    peptide = peptide.upper()
    peptide = [protein_mapping[aa] for aa in peptide]
    peptide = np.asarray(peptide, dtype=np.float32)

    # ensure shape is (1, 1, 1000, 9), padding with zeros if necessary
    if peptide.shape[0] < 1000:
        peptide = np.concatenate((peptide, np.zeros((1000 - peptide.shape[0], 9))), axis=0)
    elif peptide.shape[0] > 1000:
        peptide = peptide[:1000]

    peptide = torch.tensor(peptide).unsqueeze(0).unsqueeze(0)
    return peptide

def encode_nucleotide_sequence(nucleotide_sequence):
    nucleotide_sequence = nucleotide_sequence.upper()
    nucleotide_sequence = [nucleotide_mapping[nt] for nt in nucleotide_sequence]
    nucleotide_sequence = np.asarray(nucleotide_sequence, dtype=np.float32)

    # ensure shape is (1, 1, 75, 5), padding with zeros if necessary
    if nucleotide_sequence.shape[0] < 75:
        nucleotide_sequence = np.concatenate((nucleotide_sequence, np.zeros((75 - nucleotide_sequence.shape[0], 5))), axis=0)
    elif nucleotide_sequence.shape[0] > 75:
        nucleotide_sequence = nucleotide_sequence[:75]

    nucleotide_sequence = torch.tensor(nucleotide_sequence).unsqueeze(0).unsqueeze(0)
    return nucleotide_sequence

In [4]:
# seed random number generators
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# seed torch cuda
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# "Pre-train" on no mutation data

In [5]:
with open(f'../datasets/pronab_no_mutations.json', 'r') as f:
    train_set = json.load(f)

training_tuples = [(train_set[key]['Sequence'], # Peptide sequence
                    [item[0] for item in train_set[key]['binding data']], # Nucleotide sequence
                    [item[3] for item in train_set[key]['binding data']], # G0
                    [item[1] for item in train_set[key]['binding data']]) for key in train_set] # RNA/DNA


############ DECONTAMINATE TRAIN SET ############
test_dataset = []
with open('../datasets/mutation_data.jsonl', 'r') as f:
    for line in f:
        test_dataset.append(json.loads(line))

sequences = [test_dataset[i]["peptide_sequence"] for i in range(len(test_dataset))]

# group by peptide sequence
grouped_sequences = {}
for i in range(len(test_dataset)):
    if test_dataset[i]["peptide_sequence"] not in grouped_sequences:
        grouped_sequences[test_dataset[i]["peptide_sequence"]] = []
    grouped_sequences[test_dataset[i]["peptide_sequence"]].append(test_dataset[i])

# remove sequences that are in the training set
deleted = 0
for i in range(len(training_tuples)-1, -1, -1):
    if training_tuples[i][0] in grouped_sequences:
        del training_tuples[i]
        deleted += 1
print(f"Deleted {deleted} proteins from the train set")
###############################################

peptide_sequences = []
nucleotide_sequences = []
G0 = []
for i in range(len(training_tuples)):
    for j in range(len(training_tuples[i][1])):
        nucleotide_sequence = training_tuples[i][1][j]

        if len(nucleotide_sequence) == 0:
            continue

        if training_tuples[i][3][j] == "RNA":
            nucleotide_sequence.replace("T", "U")

        peptide_sequences.append(training_tuples[i][0])
        nucleotide_sequences.append(nucleotide_sequence)
        G0.append(training_tuples[i][2][j])

G0_mean = np.mean(G0)
G0_std = np.std(G0)

Deleted 92 proteins from the train set


In [7]:
len(peptide_sequences), len(set(peptide_sequences))

(10882, 660)

In [6]:
num_epochs = 32
batch_size = 256

model = DeePNAP().to(device).to(dtype)
model.train()

num_steps = int(num_epochs*len(peptide_sequences)/batch_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.0, total_iters=num_steps)

pbar = tqdm(range(num_steps))
for step in pbar:
    optimizer.zero_grad()

    if len(nucleotide_sequences[i]) == 0 or len(peptide_sequences[i]) == 0:
        continue

    indices = np.random.choice(len(peptide_sequences), batch_size, replace=False)

    X_protein = [encode_peptide(peptide_sequences[i]) for i in indices]
    X_wild = [encode_nucleotide_sequence(nucleotide_sequences[i]) for i in indices]

    X_protein = torch.cat(X_protein, dim=0).to(device, dtype)
    X_wild = torch.cat(X_wild, dim=0).to(device, dtype)

    G0_wild_ground_truths = torch.tensor([(G0[i] - G0_mean) / G0_std for i in indices], dtype=dtype).to(device)
    G0_wild = model(X_protein, X_wild).reshape(-1) * G0_std + G0_mean

\
    loss = ((G0_wild - G0_wild_ground_truths)**2).mean()
    loss.backward()
    
    optimizer.step()
    scheduler.step()

    pbar.set_description(f"Loss: {loss:.4f}")

Loss: 0.5643: 100%|██████████| 1360/1360 [03:09<00:00,  7.17it/s]


# Evaluate Pre-trained model

In [7]:
base_model = copy.deepcopy(model)

test_dataset = []
with open('../datasets/mutation_data.jsonl', 'r') as f:
    for line in f:
        test_dataset.append(json.loads(line))

sequences = [test_dataset[i]["peptide_sequence"] for i in range(len(test_dataset))]

random.shuffle(sequences)

# group by peptide sequence
grouped_sequences = {}
for i in range(len(test_dataset)):
    if test_dataset[i]["peptide_sequence"] not in grouped_sequences:
        grouped_sequences[test_dataset[i]["peptide_sequence"]] = []
    grouped_sequences[test_dataset[i]["peptide_sequence"]].append(test_dataset[i])

last_test_pcc = 0.0
MAE = 0.0

last_dG_pcc = 0.0
dG_MAE = 0.0

all_test_pccs = []
all_MAEs = []

all_dG_pccs = []
all_dG_MAEs = []

test_set = []
for i, key in enumerate(grouped_sequences.keys()):
    test_set += grouped_sequences[key]

model = copy.deepcopy(base_model)

with torch.no_grad():
    model.eval()
    
    ground_truths = []
    predictions = []

    dG_predictions = []
    dG_ground_truths = []
    
    for i in range(0, len(test_set)):
        X_protein = encode_peptide(test_set[i]["peptide_sequence"]).to(device, dtype)
        X_wild = encode_nucleotide_sequence(test_set[i]["wild_nucleotide_sequence"]).to(device, dtype)
        X_mutated = encode_nucleotide_sequence(test_set[i]["mutated_nucleotide_sequence"]).to(device, dtype)

        G0_wild = model(X_protein, X_wild).reshape(-1).item() * G0_std + G0_mean
        G0_mutated = model(X_protein, X_mutated).reshape(-1).item() * G0_std + G0_mean

        dG_predictions.extend([G0_wild, G0_mutated])
        dG_ground_truths.extend([test_set[i]["wild_G0"], test_set[i]["mutant_G0"]])

        difference = G0_mutated - G0_wild
        ground_truth_difference = test_set[i]["mutant_G0"] - test_set[i]["wild_G0"]

        ground_truths.append(ground_truth_difference)
        predictions.append(difference)

    ground_truths = np.asarray(ground_truths)
    predictions = np.asarray(predictions)

    dG_ground_truths = np.asarray(dG_ground_truths)
    dG_predictions = np.asarray(dG_predictions)

    last_test_pcc = pearsonr(ground_truths, predictions)[0]
    MAE = np.abs(ground_truths - predictions).mean()

    last_dG_pcc = pearsonr(dG_ground_truths, dG_predictions)[0]
    dG_MAE = np.abs(dG_ground_truths - dG_predictions).mean()

    all_test_pccs.append(last_test_pcc)
    all_MAEs.append(MAE)

    all_dG_pccs.append(last_dG_pcc)
    all_dG_MAEs.append(dG_MAE)

    # print results
    print(f"Test PCC: {last_test_pcc:.4f}")
    print(f"Test MAE: {MAE:.4f}")
    print(f"dG PCC: {last_dG_pcc:.4f}")
    print(f"dG MAE: {dG_MAE:.4f}")
    print("=============================================")

Test PCC: 0.0134
Test MAE: 1.0658
dG PCC: 0.2178
dG MAE: 9.5376


# Train on Mutation data

In [8]:
base_model = copy.deepcopy(model)

num_epochs = 256
batch_size = 256

test_dataset = []
with open('../datasets/mutation_data.jsonl', 'r') as f:
    for line in f:
        test_dataset.append(json.loads(line))

sequences = [test_dataset[i]["peptide_sequence"] for i in range(len(test_dataset))]

random.shuffle(sequences)

# group by peptide sequence
grouped_sequences = {}
for i in range(len(test_dataset)):
    if test_dataset[i]["peptide_sequence"] not in grouped_sequences:
        grouped_sequences[test_dataset[i]["peptide_sequence"]] = []
    grouped_sequences[test_dataset[i]["peptide_sequence"]].append(test_dataset[i])

last_test_pcc = 0.0
MAE = 0.0

last_dG_pcc = 0.0
dG_MAE = 0.0

all_test_pccs = []
all_MAEs = []

all_dG_pccs = []
all_dG_MAEs = []

for split in range(10):
    train_set = []
    test_set = []
    for i, key in enumerate(grouped_sequences.keys()):
        if i % 10 == split:
            test_set += grouped_sequences[key]
        train_set += grouped_sequences[key]

    model = copy.deepcopy(base_model)
    model.train()

    num_steps = int(num_epochs*len(train_set)/batch_size)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    #scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.0, total_iters=num_steps)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, total_steps=num_steps, pct_start=0.05)

    pbar = tqdm(range(num_steps))
    for step in pbar:
        optimizer.zero_grad()

        indices = np.random.choice(len(train_set), batch_size, replace=False)

        X_protein = [encode_peptide(train_set[i]["peptide_sequence"]) for i in indices]
        X_wild = [encode_nucleotide_sequence(train_set[i]["wild_nucleotide_sequence"]) for i in indices]
        X_mutated = [encode_nucleotide_sequence(train_set[i]["mutated_nucleotide_sequence"]) for i in indices]

        X_protein = torch.cat(X_protein, dim=0).to(device, dtype)
        X_wild = torch.cat(X_wild, dim=0).to(device, dtype)
        X_mutated = torch.cat(X_mutated, dim=0).to(device, dtype)

        G0_wild_ground_truths = torch.tensor([train_set[i]["wild_G0"] for i in indices], dtype=dtype).to(device)
        G0_mutated_ground_truths = torch.tensor([train_set[i]["mutant_G0"] for i in indices], dtype=dtype).to(device)
        G0_diff_ground_truths = G0_mutated_ground_truths - G0_wild_ground_truths

        G0_wild = model(X_protein, X_wild).reshape(-1) * G0_std + G0_mean
        G0_mutated = model(X_protein, X_mutated).reshape(-1) * G0_std + G0_mean

        G0_diff = G0_mutated - G0_wild

        loss = ((G0_diff - G0_diff_ground_truths)**2).mean() + ((G0_wild - G0_wild_ground_truths)**2).mean() + ((G0_mutated - G0_mutated_ground_truths)**2).mean()
        loss.backward()
        
        optimizer.step()
        scheduler.step()

        pbar.set_description(f"Loss: {loss:.4f}, last pcc: {last_test_pcc:.4f}, MAE: {MAE:.4f}, dG pcc: {last_dG_pcc:.4f}, dG MAE: {dG_MAE:.4f}")
        
        if step % 100 == 0:
            with torch.no_grad():
                model.eval()
                ground_truths = []
                predictions = []

                dG_predictions = []
                dG_ground_truths = []

                for i in range(0, len(test_set)):
                    X_protein = encode_peptide(test_set[i]["peptide_sequence"]).to(device, dtype)
                    X_wild = encode_nucleotide_sequence(test_set[i]["wild_nucleotide_sequence"]).to(device, dtype)
                    X_mutated = encode_nucleotide_sequence(test_set[i]["mutated_nucleotide_sequence"]).to(device, dtype)
                    
                    G0_wild = model(X_protein, X_wild).reshape(-1).item() * G0_std + G0_mean
                    G0_mutated = model(X_protein, X_mutated).reshape(-1).item() * G0_std + G0_mean

                    dG_predictions.extend([G0_wild, G0_mutated])
                    dG_ground_truths.extend([test_set[i]["wild_G0"], test_set[i]["mutant_G0"]])

                    difference = G0_mutated - G0_wild
                    ground_truth_difference = test_set[i]["mutant_G0"] - test_set[i]["wild_G0"]

                    ground_truths.append(ground_truth_difference)
                    predictions.append(difference)

                    pbar.set_description(f"(Testing {i}/{len(test_set)}) Loss: {loss:.4f}, test pcc: {last_test_pcc:.4f}, test MAE: {MAE:.4f}")

                ground_truths = np.asarray(ground_truths)
                predictions = np.asarray(predictions)

                dG_ground_truths = np.asarray(dG_ground_truths)
                dG_predictions = np.asarray(dG_predictions)

                last_test_pcc = pearsonr(ground_truths, predictions)[0]
                MAE = np.abs(ground_truths - predictions).mean()

                last_dG_pcc = pearsonr(dG_ground_truths, dG_predictions)[0]
                dG_MAE = np.abs(dG_ground_truths - dG_predictions).mean()

                model.train()
    
    with torch.no_grad():
        model.eval()
        
        ground_truths = []
        predictions = []

        dG_predictions = []
        dG_ground_truths = []
        
        for i in range(0, len(test_set)):
            X_protein = encode_peptide(test_set[i]["peptide_sequence"]).to(device, dtype)
            X_wild = encode_nucleotide_sequence(test_set[i]["wild_nucleotide_sequence"]).to(device, dtype)
            X_mutated = encode_nucleotide_sequence(test_set[i]["mutated_nucleotide_sequence"]).to(device, dtype)

            G0_wild = model(X_protein, X_wild).reshape(-1).item() * G0_std + G0_mean
            G0_mutated = model(X_protein, X_mutated).reshape(-1).item() * G0_std + G0_mean

            dG_predictions.extend([G0_wild, G0_mutated])
            dG_ground_truths.extend([test_set[i]["wild_G0"], test_set[i]["mutant_G0"]])

            difference = G0_mutated - G0_wild
            ground_truth_difference = test_set[i]["mutant_G0"] - test_set[i]["wild_G0"]

            ground_truths.append(ground_truth_difference)
            predictions.append(difference)

        ground_truths = np.asarray(ground_truths)
        predictions = np.asarray(predictions)

        dG_ground_truths = np.asarray(dG_ground_truths)
        dG_predictions = np.asarray(dG_predictions)

        last_test_pcc = pearsonr(ground_truths, predictions)[0]
        MAE = np.abs(ground_truths - predictions).mean()

        last_dG_pcc = pearsonr(dG_ground_truths, dG_predictions)[0]
        dG_MAE = np.abs(dG_ground_truths - dG_predictions).mean()

        all_test_pccs.append(last_test_pcc)
        all_MAEs.append(MAE)

        all_dG_pccs.append(last_dG_pcc)
        all_dG_MAEs.append(dG_MAE)

        # print results
        print(f"Split {split}:")
        print(f"Test PCC: {last_test_pcc:.4f}")
        print(f"Test MAE: {MAE:.4f}")
        print(f"dG PCC: {last_dG_pcc:.4f}")
        print(f"dG MAE: {dG_MAE:.4f}")
        print("=============================================")

Loss: 4.3251, last pcc: 0.1645, MAE: 0.6162, dG pcc: 0.5514, dG MAE: 0.5768: 100%|██████████| 706/706 [01:54<00:00,  6.15it/s]   


Split 0:
Test PCC: 0.1645
Test MAE: 0.6162
dG PCC: 0.5514
dG MAE: 0.5768


Loss: 4.8128, last pcc: 0.2537, MAE: 0.8232, dG pcc: 0.9255, dG MAE: 0.4986: 100%|██████████| 706/706 [01:57<00:00,  6.03it/s]   


Split 1:
Test PCC: 0.2537
Test MAE: 0.8232
dG PCC: 0.9255
dG MAE: 0.4986


Loss: 3.5676, last pcc: 0.3426, MAE: 1.2458, dG pcc: 0.8923, dG MAE: 0.7201: 100%|██████████| 706/706 [01:54<00:00,  6.18it/s]  


...

Split 7:
Test PCC: 0.7047
Test MAE: 0.9525
dG PCC: 0.8641
dG MAE: 0.5649


Loss: 4.6555, last pcc: 0.2815, MAE: 0.9943, dG pcc: 0.8914, dG MAE: 0.6115: 100%|██████████| 706/706 [02:07<00:00,  5.55it/s]   


Split 8:
Test PCC: 0.2815
Test MAE: 0.9943
dG PCC: 0.8914
dG MAE: 0.6115


Loss: 3.3119, last pcc: 0.4261, MAE: 0.8794, dG pcc: 0.9284, dG MAE: 0.4874: 100%|██████████| 706/706 [02:09<00:00,  5.47it/s]  


Split 9:
Test PCC: 0.4261
Test MAE: 0.8794
dG PCC: 0.9284
dG MAE: 0.4873


In [9]:
print(f"Average test PCC: {np.mean(all_test_pccs):.4f} ± {np.std(all_test_pccs)/np.sqrt(10):.4f}")
print(f"Average test MAE: {np.mean(all_MAEs):.4f} ± {np.std(all_MAEs)/np.sqrt(10):.4f}")

print(f"Average dG PCC: {np.mean(all_dG_pccs):.4f} ± {np.std(all_dG_pccs)/np.sqrt(10):.4f}")
print(f"Average dG MAE: {np.mean(all_dG_MAEs):.4f} ± {np.std(all_dG_MAEs)/np.sqrt(10):.4f}")

Average test PCC: 0.3918 ± 0.0511
Average test MAE: 0.9647 ± 0.0638
Average dG PCC: 0.8250 ± 0.0435
Average dG MAE: 0.6410 ± 0.0514


In [11]:
with open(f'../datasets/pronab_no_mutations.json', 'r') as f:
    train_set = json.load(f)

training_tuples = [(train_set[key]['Sequence'], # Peptide sequence
                    [item[0] for item in train_set[key]['binding data']], # Nucleotide sequence
                    [item[3] for item in train_set[key]['binding data']], # G0
                    [item[1] for item in train_set[key]['binding data']]) for key in train_set] # RNA/DNA


############ DECONTAMINATE TRAIN SET ############
test_dataset = []
with open('../datasets/mutation_data.jsonl', 'r') as f:
    for line in f:
        test_dataset.append(json.loads(line))

sequences = [test_dataset[i]["peptide_sequence"] for i in range(len(test_dataset))]

# group by peptide sequence
grouped_sequences = {}
for i in range(len(test_dataset)):
    if test_dataset[i]["peptide_sequence"] not in grouped_sequences:
        grouped_sequences[test_dataset[i]["peptide_sequence"]] = []
    grouped_sequences[test_dataset[i]["peptide_sequence"]].append(test_dataset[i])

# remove sequences that are in the training set
deleted = 0
for i in range(len(training_tuples)-1, -1, -1):
    if training_tuples[i][0] in grouped_sequences:
        del training_tuples[i]
        deleted += 1
print(f"Deleted {deleted} proteins from the train set")
###############################################

peptide_sequences = []
nucleotide_sequences = []
G0 = []
for i in range(len(training_tuples)):
    for j in range(len(training_tuples[i][1])):
        nucleotide_sequence = training_tuples[i][1][j]

        if len(nucleotide_sequence) == 0:
            continue

        if training_tuples[i][3][j] == "RNA":
            nucleotide_sequence.replace("T", "U")

        peptide_sequences.append(training_tuples[i][0])
        nucleotide_sequences.append(nucleotide_sequence)
        G0.append(training_tuples[i][2][j])

G0_mean = np.mean(G0)
G0_std = np.std(G0)

Deleted 92 proteins from the train set


In [16]:
keys = list(train_set.keys())

In [34]:
total_interactions = 0

for key in keys:
    total_interactions += len(train_set[key]['binding data'])

len(keys), total_interactions

(757, 14582)

In [35]:
test_sequences = [test_dataset[i]['peptide_sequence'] for i in range(len(test_dataset))]
len(set(test_sequences)), len(test_sequences)*2

(93, 1412)

In [36]:
757 + 93

850

In [37]:
14582+1412

15994